# Using EcoFOCIpy to process raw field data

## DY2306 (Spring Mooring Cruise - Dyson)

## CTD / BTL Data

Basic workflow for each instrument grouping is *(initial archive level)*:
- SBE workflow must happen first
- Parse data from btl files into pandas dataframe

Convert to xarray dataframe for all following work *(working or final data level):
- Add metadata from cruise yaml files and/or header info
- ingest metadata from cruise / cast logs
- process data beyond simple file translate
- apply any calibrations or corrections
    + field corrections
    + offsets
    + instrument compensations
    + some QC were available... this would be old-school simple bounds mostly
- adjust time bounds and sample frequency (xarray dataframe)
- save as CF netcdf via xarray: so many of the steps above are optional
    + **ERDDAP NRT** if no corrections, offsets or time bounds are applied but some meta data is
    + **Working and awaiting QC** has no ERDDAP representation and is a holding spot
    + **ERDDAP Final** fully calibrated, qc'd and populated with meta information

Plot for preview and QC
- preview images (indiv and/or collectively)
- manual qc process
- automated qc process ML/AI

Further refinenments for ERDDAP hosting:

## Differences from EPIC standard
- previously btl files had coordinates of lat/lon/time/depth...with a bottle position / fireing order being a variable, this means that if multiple bottles are fired at the same depth, the file was not uniquely indexed and the bottle variable had to be conflated for multiple fireings
- in new version, we will index based on lat/lon/time/bottle_num (bottle number is a sequential unique value... often representing the position on the rosette), merging with CTD downcast data will require maintining a pressure/depth variable in the bottle data that can be rounded to the nearest 1m bin.  This does not solve the problem with multiple discrete samples taken at a single depth and single niskin though.

## Example below is for SBE 9/11+ V2 but the workflow is similar for any SBE instruments.

Future processing of this instrument can be a simplified (no markdown) process which can be archived so that the procedure can be traced or updated

We process each cast as an individual file so this example will not loop over all profiles.  See `example/all_casts.py` example for processing an entire cruise at once.

Adding Discrete samples such as Oxygen, Chlorophyll, Salinity to BTL Data is in `example/discrete_castdata.py`.  Its purpose is to match niskin/bottle information to depth for the discrete data.

In [1]:
import yaml
import glob

import ecofocipy.io.sbe_ctd_parser as sbe_ctd_parser #<- instrument specific
import ecofocipy.io.ncCFsave as ncCFsave
import ecofocipy.metaconfig.load_config as load_config

The sample_data_dir should be included in the github package but may not be included in the pip install of the package

## Simple Processing - first step

In [2]:
sample_data_dir = '/Users/bell/ecoraid/2023/CTDcasts/dy2306/' #root path to cruise directory
ecofocipy_dir = '/Users/bell/Programs/EcoFOCIpy/'

In [7]:
###############################################################
# edit to point to {cruise sepcific} raw datafiles 
datafile = sample_data_dir+'rawconverted/' #<- point to cruise and process all files within
cruise_name = 'DY2306' #no hyphens
cruise_meta_file = sample_data_dir+'logs/DY2306.yaml'
inst_meta_file = sample_data_dir+'logs/FOCI_standard_CTD.yaml' #<- copy to each deployment for simplicity?
group_meta_file = ecofocipy_dir+'staticdata/institutional_meta_example.yaml'
###############################################################

#init and load data
cruise = sbe_ctd_parser.sbe_btl()
filename_list = sorted(glob.glob(datafile + '*.btl'))

cruise_data = cruise.manual_parse(filename_list)

Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd001.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd003.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd004.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd005.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd006.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd007.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd008.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd009.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd010.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd011.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd012.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd013.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd014.btl
Processing /

/Users/bell/Programs/EcoFOCIpy/src/ecofocipy/io/sbe_ctd_parser.py:90: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  ctd_df['datetime']=pd.to_datetime(ctd_df['date']+' '+ctd_df['time'])


Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd059.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd060.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd061.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd065.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd066.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd067.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd068.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd069.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd070.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd071.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd072.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd073.btl
Processing /Users/bell/ecoraid/2023/CTDcasts/dy2306/rawconverted/ctd074.btl
Processing /

In [8]:
#quick statistical look at the distribution of data for a cast
# #preview a dataframe
cruise_data['ctd001.btl'].describe()

,sbeox0ml/l,sbeox0ps,sbox0mm/kg,sal00,sal11,sigma-t00,sigma-t11,sbeox1ml/l,sbeox1ps,sbox1mm/kg,fleco-afl,t090c,t190c,prdm,turbwetntu0,par,scan,datetime
count,1.0000,1.000,1.000,1.0000,1.000,1.0000,1.0000,1.0000,1.000,1.000,1.0000,1.0000,1.0000,1.000,1.000,1.000000e+00,1.0,1
mean,8.7849,117.942,382.652,31.8459,31.848,25.3029,25.3046,7.4644,100.214,325.135,2.3597,3.7358,3.7355,65.421,0.547,1.000000e-12,4527.0,2023-04-22 22:33:24
min,8.7849,117.942,382.652,31.8459,31.848,25.3029,25.3046,7.4644,100.214,325.135,2.3597,3.7358,3.7355,65.421,0.547,1.000000e-12,4527.0,2023-04-22 22:33:24
25%,8.7849,117.942,382.652,31.8459,31.848,25.3029,25.3046,7.4644,100.214,325.135,2.3597,3.7358,3.7355,65.421,0.547,1.000000e-12,4527.0,2023-04-22 22:33:24
50%,8.7849,117.942,382.652,31.8459,31.848,25.3029,25.3046,7.4644,100.214,325.135,2.3597,3.7358,3.7355,65.421,0.547,1.000000e-12,4527.0,2023-04-22 22:33:24
75%,8.7849,117.942,382.652,31.8459,31.848,25.3029,25.3046,7.4644,100.214,325.135,2.3597,3.7358,3.7355,65.421,0.547,1.000000e-12,4527.0,2023-04-22 22:33:24
max,8.7849,117.942,382.652,31.8459,31.848,25.3029,25.3046,7.4644,100.214,325.135,2.3597,3.7358,3.7355,65.421,0.547,1.000000e-12,4527.0,2023-04-22 22:33:24
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Create BTL report file

In [9]:
# btl report file
for cast in cruise_data.keys():
    try:
        df=cruise_data[cast]
        df['cast'] = cast.lower().split('.')[0]
        if cast.lower().split('.')[0] == 'ctd001':
            df.to_csv(f'{cruise_name}.report_btl')
        else:
            df.to_csv(f'{cruise_name}.report_btl',mode='a', header=False)
    except:
        print(f'some issue in {cast}')

## Time Properties

Not traditionally dealt with for CTD files as they are likely dynamically updated via GPS feed.  However, FOCI tends to label the date/time with the ***at depth*** time-stamp

## Depth Properties and other assumptions

- currently, all processing and binning (1m for FOCI) is done via seabird routines and the windows software.  This may change with the python ctd package for a few tasks

## Add Deployment meta information

In [10]:
#just a dictionary of dictionaries - simple
with open(cruise_meta_file) as file:
    cruise_config = yaml.full_load(file)
cruise_config[cruise_name]

{'CruiseID': 'DY2306',
 'CruiseID_Historic': '',
 'CruiseID_Alternates': '',
 'Project_Leg': '',
 'Vessel': 'NOAAS Oscar Dyson',
 'ShipID': 'DY',
 'StartDate': datetime.date(2023, 4, 21),
 'EndDate': datetime.date(2022, 5, 7),
 'Project': 'FOCI',
 'ChiefScientist': 'Shaun Bell',
 'StartPort': 'Kodiak, AK',
 'EndPort': 'Dutch Harbor, AK',
 'CruiseLocation': 'Bering Sea',
 'Description': 'FOCI Spring Mooring Survey',
 'CruiseYear': 2023,
 'ctdlogs_pdf_name': 'DY2306_CTDCastLogs.pdf'}

In [11]:
#and if you want a cast from the cruise, just use the consective cast number
cruise_config['CTDCasts']['CTD001']

{'id': 54144,
 'Vessel': 'NOAAS Oscar Dyson',
 'CruiseID': 'DY2306',
 'Project_Leg': '',
 'UniqueCruiseID': 'DY2306',
 'Project': 'EcoFOCI Spring Mooring Cruise',
 'StationNo_altname': 's1h1',
 'ConsecutiveCastNo': 'CTD001',
 'LatitudeDeg': 54,
 'LatitudeMin': 50.17,
 'LongitudeDeg': 158,
 'LongitudeMin': 58.97,
 'GMTDay': 23,
 'GMTMonth': 'Apr',
 'GMTYear': 2023,
 'GMTTime': 23580,
 'DryBulb': 5.5,
 'RelativeHumidity': 54,
 'WetBulb': -99.9,
 'Pressure': 1017,
 'SeaState': '',
 'Visibility': '',
 'WindDir': 309,
 'WindSpd': 24.0,
 'CloudAmt': '',
 'CloudType': '',
 'Weather': '',
 'SurfaceTemp': -99.9,
 'BottomDepth': 74,
 'StationNameID': 'Shumagins',
 'MaxDepth': 65,
 'InstrumentSerialNos': '',
 'Notes': '',
 'NutrientBtlNiskinNo': '',
 'NutrientBtlNumbers': '',
 'OxygenBtlNiskinNo': '1',
 'OxygenBtlNumbers': '244',
 'SalinityBtlNiskinNo': '1',
 'SalinityBtlNumbers': '121',
 'ChlorophyllBtlNiskinNo': '',
 'ChlorophyllBtlVolumes': '',
 'InstrumentType': 'SBE911plus',
 'WaterMassCode'

## Add Instrument meta information

Time, depth, lat, lon should be added regardless (always our coordinates) but for a mooring site its going to be a (1,1,1,t) dataset
The variables of interest should be read from the data file and matched to a key for naming.  That key is in the inst_config file seen below and should represent common conversion names in the raw data

In [12]:
with open(inst_meta_file) as file:
    inst_config = yaml.full_load(file)
inst_config

{'time': {'epic_key': 'TIM_601',
  'name': 'time',
  'generic_name': 'time',
  'standard_name': 'time',
  'long_name': 'date and time since reference time'},
 'depth': {'epic_key': 'D_3',
  'generic_name': 'depth',
  'units': 'meter',
  'long_name': 'depth below surface (meters)',
  'standard_name': 'depth',
  'positive': 'down'},
 'latitude': {'epic_key': 'LON_501',
  'name': 'latitude',
  'generic_name': 'latitude',
  'units': 'degrees_north',
  'long_name': 'latitude',
  'standard_name': 'latitude'},
 'longitude': {'epic_key': 'LAT_500',
  'name': 'longitude',
  'generic_name': 'longitude',
  'units': 'degrees_east',
  'long_name': 'longitude',
  'standard_name': 'longitude'},
 'temperature_ch1': {'epic_key': 'T_28',
  'generic_name': 'temp channel 1',
  'long_name': 'Sea temperature in-situ ITS-90 scale',
  'standard_name': 'sea_water_temperature',
  'units': 'degree_C'},
 'temperature_ch2': {'epic_key': 'T2_35',
  'generic_name': 'temp channel 2',
  'long_name': 'Sea temperature i

In [13]:
#sbe data uses header info to name variables... but we want standard names from the dictionary I've created, so we need to rename column variables appropriately
#rename values to appropriate names, if a value isn't in the .yaml file, you can add it

#*** biggest *** difference between moored and profile data is there may be multiple instruments with the same dataype (e.g.) temperature
# on the same platform.  We _used_ to use the phrases primary and secondary, but will now only refer to them as ch1, ch2 etc
cruise_data['ctd001.btl'] = cruise_data['ctd001.btl'].rename(columns={
                        't090c':'temperature_ch1',
                        't190c':'temperature_ch2',
                        'sal00':'salinity_ch1',
                        'sal11':'salinity_ch2',
                        'sbox0mm/kg':'oxy_conc_ch1',
                        'sbeox0ml/l':'oxy_concM_ch1',
                        'sbox1mm/kg':'oxy_conc_ch2',
                        'sbeox1ml/l':'oxy_concM_ch2',
                        'sbeox0ps':'oxy_percentsat_ch1',
                        'sbeox1ps':'oxy_percentsat_ch2',
                        'sigma-t00':'sigma_t_ch1',
                        'sigma-t11':'sigma_t_ch2',
                        'cstarat0':'Attenuation',
                        'cstartr0':'Transmittance',
                        'fleco-afl':'chlor_fluorescence',
                        'turbwetntu0':'turbidity',
                        'empty':'empty', #this will be ignored
                        'prdm':'Pressure [dbar]',
                        'flag':'flag'})

cruise_data['ctd001.btl'].sample()

,oxy_concM_ch1,oxy_percentsat_ch1,oxy_conc_ch1,salinity_ch1,salinity_ch2,sigma_t_ch1,sigma_t_ch2,oxy_concM_ch2,oxy_percentsat_ch2,oxy_conc_ch2,chlor_fluorescence,temperature_ch1,temperature_ch2,Pressure [dbar],turbidity,par,scan,datetime,cast
bottle,,,,,,,,,,,,,,,,,,,
1.0,8.7849,117.942,382.652,31.8459,31.848,25.3029,25.3046,7.4644,100.214,325.135,2.3597,3.7358,3.7355,65.421,0.547,1.000000e-12,4527.0,2023-04-22 22:33:24,ctd001


In [14]:
cruise_data['ctd001.btl'].columns

Index(['oxy_concM_ch1', 'oxy_percentsat_ch1', 'oxy_conc_ch1', 'salinity_ch1',
       'salinity_ch2', 'sigma_t_ch1', 'sigma_t_ch2', 'oxy_concM_ch2',
       'oxy_percentsat_ch2', 'oxy_conc_ch2', 'chlor_fluorescence',
       'temperature_ch1', 'temperature_ch2', 'Pressure [dbar]', 'turbidity',
       'par', 'scan', 'datetime', 'cast'],
      dtype='object')

## Add institutional meta-information


In [15]:
with open(group_meta_file) as file:
    group_config = yaml.full_load(file)
group_config

{'source_documents': 'http://www.oceansites.org/docs/oceansites_data_format_reference_manual.pdf',
 'institution': 'Pacific Marine Environmental Lab (PMEL)',
 'project': 'EcoFOCI',
 'project_url': 'https://www.ecofoci.noaa.gov',
 'principal_investigator': 'Phyllis Stabeno',
 'principal_investigator_email': 'phyllis.stabeno (at) noaa.gov',
 'creator_name': 'Shaun Bell',
 'creator_email': 'shaun.bell (at) noaa.gov',
 'creator_institution': 'PMEL',
 'keywords': 'Mooring, Oceanographic',
 'comment': 'Provisional data',
 'sea_area': 'Bering Sea (BS)',
 'featureType': 'timeSeries',
 'conventions': '”CF-1.6, ~OceanSITES-1.5, ACDD-1.2”',
 'license': '',
 'references': '',
 'citation': '',
 'acknowledgement': ''}

In [16]:
# Add meta data and prelim processing based on meta data
# Convert to xarray and add meta information - save as CF netcdf file
# pass -> data, instmeta, depmeta
cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data['ctd001.btl'], 
                                instrument_yaml=inst_config, 
                                operation_yaml=cruise_config,
                                operation_type='ctd')
cruise_data_nc

In [17]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 1)
Coordinates:
  * bottle              (bottle) float64 1.0
Data variables: (12/19)
    oxy_concM_ch1       (bottle) float64 8.785
    oxy_percentsat_ch1  (bottle) float64 117.9
    oxy_conc_ch1        (bottle) float64 382.7
    salinity_ch1        (bottle) float64 31.85
    salinity_ch2        (bottle) float64 31.85
    sigma_t_ch1         (bottle) float64 25.3
    ...                  ...
    Pressure [dbar]     (bottle) float64 65.42
    turbidity           (bottle) float64 0.547
    par                 (bottle) float64 1e-12
    scan                (bottle) float64 4.527e+03
    datetime            (bottle) datetime64[ns] 2023-04-22T22:33:24
    cast                (bottle) object 'ctd001'

At this point, you could save your file with the `.xarray2netcdf_save()` method and have a functioning dataset.... but it would be very simple with no additional qc, meta-data, or tuned parameters for optimizing software like ferret or erddap.

In [18]:
# expand the dimensions and coordinate variables
# renames them appropriatley and prepares them for meta-filled values
cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

In [19]:
#build list from columsn in data - if a variable isn't in the yaml file, it will be dropped from the final data fields
cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data['ctd001.btl'].columns.values),drop_missing=False)
#adding dimension meta needs to come after updating the dimension values... BUG?
cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])

The following steps can happen in just about any order and are all meta-data driven.  Therefore, they are not required to have a functioning dataset, but they are required to have a well described dataset

In [20]:
cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 1, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/19)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 8.785
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 117.9
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 382.7
    salinity_ch1        (latitude, longitude, time, bottle) float64 31.85
    salinity_ch2        (latitude, longitude, time, bottle) float64 31.85
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 25.3
    ...                  ...
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 65.42
    turbidity           (latitude, longitude, time, bottle) float64 0.547
    par                 (latitude, longitude, time, bottle) float64 1e-12
    scan                (latitude, longitude, time, bottle) float64 4.527e+03
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd001'

In [21]:
#add global attributes
cruise_data_nc.deployment_meta_add(conscastno='CTD001')

#add instituitonal global attributes
cruise_data_nc.institution_meta_add(group_config)

#add creation date/time - provenance data
cruise_data_nc.provinance_meta_add()

#provide intial qc status field
cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

cruise_data_nc.get_xdf()

<xarray.Dataset>
Dimensions:             (bottle: 1, latitude: 1, longitude: 1, time: 1)
Coordinates:
  * bottle              (bottle) float64 1.0
  * latitude            (latitude) float64 1e+35
  * longitude           (longitude) float64 1e+35
  * time                (time) float64 1e+35
Data variables: (12/19)
    oxy_concM_ch1       (latitude, longitude, time, bottle) float64 8.785
    oxy_percentsat_ch1  (latitude, longitude, time, bottle) float64 117.9
    oxy_conc_ch1        (latitude, longitude, time, bottle) float64 382.7
    salinity_ch1        (latitude, longitude, time, bottle) float64 31.85
    salinity_ch2        (latitude, longitude, time, bottle) float64 31.85
    sigma_t_ch1         (latitude, longitude, time, bottle) float64 25.3
    ...                  ...
    Pressure [dbar]     (latitude, longitude, time, bottle) float64 65.42
    turbidity           (latitude, longitude, time, bottle) float64 0.547
    par                 (latitude, longitude, time, bottle) float64 1e-12
    scan                (latitude, longitude, time, bottle) float64 4.527e+03
    datetime            (latitude, longitude, time, bottle) datetime64[ns] 20...
    cast                (latitude, longitude, time, bottle) object 'ctd001'
Attributes: (12/44)
    CruiseID:                      DY2306
    CruiseID_Historic:             
    CruiseID_Alternates:           
    Project_Leg:                   
    Vessel:                        NOAAS Oscar Dyson
    ShipID:                        DY
    ...                            ...
    references:                    
    citation:                      
    acknowledgement:               
    date_created:                  2023-05-30T15:29:15Z
    date_modified:                 
    QC_indicator:                  unqcd

## Save CF Netcdf files

Currently stick to netcdf3 classic... but migrating to netcdf4 (default) may be no problems for most modern purposes.  Its easy enough to pass the `format` kwargs through to the netcdf api of xarray.

In [22]:
#loop over all casts and perform tasks shown above

for cast in cruise_data.keys():
    try:
        cruise_data[cast] = cruise_data[cast].rename(columns={
                            't090c':'temperature_ch1',
                            't190c':'temperature_ch2',
                            'sal00':'salinity_ch1',
                            'sal11':'salinity_ch2',
                            'sbox0mm/kg':'oxy_conc_ch1',
                            'sbeox0ml/l':'oxy_concM_ch1',
                            'sbox1mm/kg':'oxy_conc_ch2',
                            'sbeox1ml/l':'oxy_concM_ch2',
                            'sbeox0ps':'oxy_percentsat_ch1',
                            'sbeox1ps':'oxy_percentsat_ch2',
                            'sigma-t00':'sigma_t_ch1',
                            'sigma-t11':'sigma_t_ch2',
                            'cstarat0':'Attenuation',
                            'cstartr0':'Transmittance',
                            'fleco-afl':'chlor_fluorescence',
                            'turbwetntu0':'turbidity',
                            'prdm':'pressure',
                            'empty':'empty', #this will be ignored
                            'flag':'flag'})

        cruise_data_nc = ncCFsave.EcoFOCI_CFnc(df=cruise_data[cast], 
                                    instrument_yaml=inst_config, 
                                    operation_yaml=cruise_config,
                                    operation_type='ctd')

        cruise_data_nc.expand_dimensions(dim_names=['latitude','longitude','time'],geophys_sort=False)

        cruise_data_nc.variable_meta_data(variable_keys=list(cruise_data[cast].columns.values),drop_missing=False)
        #adding dimension meta needs to come after updating the dimension values... BUG?
        cruise_data_nc.dimension_meta_data(variable_keys=['time','latitude','longitude'])
        cruise_data_nc.temporal_geospatioal_meta_data_ctd(positiveE=False,conscastno=cast.split('.')[0])

        #add global attributes
        cruise_data_nc.deployment_meta_add(conscastno=cast.split('.')[0].upper())

        #add instituitonal global attributes
        cruise_data_nc.institution_meta_add(group_config)

        #add creation date/time - provenance data
        cruise_data_nc.provinance_meta_add()

        #provide intial qc status field
        cruise_data_nc.qc_status(qc_status='unqcd') #<- options are unknown, excellent, probably good, mixed, unqcd

        cast_label = cast.lower().split('d')[-1].split('.')[0]
        cruise_data_nc.xarray2netcdf_save(xdf = cruise_data_nc.get_xdf(),
                                   filename=cruise_name+'c'+cast_label.zfill(3)+'_btl.nc',format="NETCDF3_CLASSIC")

  
    except:
        print(f'Skipping {cast}')

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history